In [ ]:
# ----------------------DNN TUNNING -------------------------------

import pandas as pd
import numpy as np
import keras_tuner as kt
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
import tensorflow as tf

# ---------------------- FOCAL LOSS ----------------------
def sparse_categorical_focal_loss(gamma=2.0, alpha=0.25):
    def loss_fn(y_true, y_pred):
        y_true = tf.cast(y_true, tf.int32)
        y_true_one_hot = tf.one_hot(y_true, depth=y_pred.shape[-1])
        cross_entropy = K.categorical_crossentropy(y_true_one_hot, y_pred)
        pt = tf.reduce_sum(y_true_one_hot * y_pred, axis=-1)
        focal_loss = alpha * tf.pow(1. - pt, gamma) * cross_entropy
        return focal_loss
    return loss_fn

# ---------------------- DATA LOADING & PREPROCESSING ----------------------
attack_free = pd.read_csv("Attack_free new.csv")[0:2369397]
dos = pd.read_csv("DoS_Attack_new.csv")[0:656578]
fuzzy = pd.read_csv("Fuzzy_Attack_New.csv")[0:591989]
impersonation = pd.read_csv("Impersonation_Attack_New.csv")[0:995471]

attack_free['label'] = 0
dos['label'] = 1
fuzzy['label'] = 2
impersonation['label'] = 3

df = pd.concat([attack_free, dos, fuzzy, impersonation], ignore_index=True)
df = df.drop(columns=[col for col in df.columns if 'Unnamed' in col])

for col in df.columns[:-1]:
    df[col] = df[col].apply(lambda x: int(str(x), 16) if isinstance(x, str) and all(c in '0123456789abcdefABCDEF' for c in str(x)) else x)
    df[col] = pd.to_numeric(df[col], errors='coerce')

df.dropna(inplace=True)

X = df.drop(columns='label').values
y = df['label'].values

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.25, random_state=42)

cw = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(cw))
input_dim = X_train.shape[1]

# ---------------------- DNN HYPERMODEL ----------------------
def build_dnn_model(hp, input_dim):
    model = Sequential()
    model.add(Input(shape=(input_dim,)))

    model.add(Dense(units=hp.Int('units_1', 128, 512, step=64), activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Float('dropout_1', 0.2, 0.5, step=0.1)))

    model.add(Dense(units=hp.Int('units_2', 64, 256, step=64), activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Float('dropout_2', 0.2, 0.5, step=0.1)))

    model.add(Dense(units=hp.Int('units_3', 32, 128, step=32), activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Float('dropout_3', 0.2, 0.5, step=0.1)))

    model.add(Dense(units=hp.Int('units_4', 32, 128, step=32), activation='relu'))
    model.add(Dropout(hp.Float('dropout_4', 0.2, 0.5, step=0.1)))

    model.add(Dense(units=hp.Int('units_5', 16, 64, step=16), activation='relu'))
    model.add(Dropout(hp.Float('dropout_5', 0.2, 0.5, step=0.1)))

    model.add(Dense(4, activation='softmax'))

    model.compile(
        optimizer=Adam(learning_rate=hp.Choice('lr', [1e-2, 1e-3, 1e-4])),
        loss=sparse_categorical_focal_loss(gamma=2.0, alpha=0.25),
        metrics=['accuracy']
    )
    return model

# ---------------------- HYPERBAND TUNING ----------------------
tuner = kt.Hyperband(
    hypermodel=lambda hp: build_dnn_model(hp, input_dim),
    objective='val_accuracy',
    max_epochs=100,
    factor=3,
    directory='dnn_5layer_tuning',
    project_name='CAN_IDS_5Layer_DNN'
)

early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

tuner.search(
    X_train, y_train,
    validation_split=0.2,
    epochs=100,
    class_weight=class_weights,
    callbacks=[early_stop], 
    verbose=1
)

# ---------------------- BEST MODEL TRAINING ----------------------
best_hps = tuner.get_best_hyperparameters(1)[0]
print("\n🔧 Best Hyperparameters Found:")
for param in best_hps.values:
    print(f"{param}: {best_hps.get(param)}")

best_model = tuner.hypermodel.build(best_hps)

start_train = time.time()
history = best_model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=best_hps.get('batch_size', 64),
    class_weight=class_weights,
    callbacks=[early_stop],
    verbose=1
)
end_train = time.time()

# ---------------------- EVALUATION ----------------------
start_test = time.time()
loss, acc = best_model.evaluate(X_test, y_test, verbose=0)
end_test = time.time()

print(f"\n✅ Tuned DNN Accuracy: {acc:.4f}")
print(f"🕒 Training Time: {end_train - start_train:.2f}s")
print(f"🧪 Testing Time: {end_test - start_test:.2f}s")

best_model.save("tuned_5layer_dnn_model.h5")